SPORTS RADAR API COMPETITIONS DATA

In [ ]:
pip install requests

In [1]:
#COLLECTING JSON COMPETITIONS DATA FROM SPORTS RADAR API
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=UMjToJVdpV74IvFAz1VrFWQcVMwv510g66U5yqOI"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

CREATING THE CATEGORY TABLE

In [2]:
#STORING JSON DATA IN JSON_DATA VARIABLE
json_data = response.json()

In [3]:
#CREATING EMPTY LIST
category_list = []

In [4]:
#STORING THE DATA IN EMPTY LIST
for competition in json_data["competitions"]:
     category_info = {
          "category_id": competition.get("category", {}).get("id"),
          "category_name": competition.get("category", {}).get("name")
        
     }
     category_list.append(category_info)

In [5]:
#IMPORTING PANDAS FOR CREATING DATAFRAME AND CLEANING DATA
import pandas as pd

In [6]:
#CREATING DATAFRAME FOR CATEGORIES DATA
dataframe_1=pd.DataFrame(category_list)

In [ ]:
#DISPLAYING DATAFRAME
dataframe_1

In [8]:
#CHECKING NULL VALUES
dataframe_1.isnull().sum()

category_id      0
category_name    0
dtype: int64

In [20]:
#CHECKING UNIQUE VALUES
dataframe_1.nunique()
dataframe_1.nunique()

category_id      16
category_name    16
dtype: int64

In [10]:
#DROPING THE DUPLICATES AND STORING IN NEW VARIABLE
dataframe_1_1 =  dataframe_1.drop_duplicates()

In [ ]:
#DISPLAYING THE DATA FOR CHECKING 
dataframe_1_1

In [ ]:
#CONVERTING THE UNCLEANED DATAFRAME TO CSV FOR CHECKING
dataframe_1.to_csv("Categories_Table.csv",index=False)

In [ ]:
#CONVERTING THE CLEANED DATAFRAME TO CSV FILES
dataframe_1_1.to_csv("Categories1_Table.csv",index=False)

CREATING THE COMPETITIONS TABLE

In [14]:
#CREATING EMPTY LIST
competition_list = []

In [15]:
#STORING THE DATA IN EMPTY LIST
for competition in json_data["competitions"]: 
        
        
        competition_info = {
            "competition_id": competition.get("id"),
            "competition_name": competition.get("name"),
            "parent_id": competition.get("parent_id"),
            "type": competition.get("type"),
            "gender": competition.get("gender"),
            "category_id": competition.get("category", {}).get("id")
        
        }

        competition_list.append(competition_info)

In [16]:
#CREATING DATAFRAME FOR COMPETITIONS DATA
dataframe_2 = pd.DataFrame(competition_list)

In [ ]:
#DISPLAYING DATAFRAME
dataframe_2

In [35]:
#CHECKING NULL VALUES
dataframe_2.isnull().sum()



competition_id       0
competition_name     0
parent_id           40
type                 0
gender               0
category_id          0
dtype: int64

In [18]:
#FILLING THE NULL VALUE NA WITH DOESN'T AFFECT THE DATA 
dataframe_2.fillna('NA',inplace=True)

In [19]:
#COUNTING THE DATA
dataframe_2.count()

competition_id      5800
competition_name    5800
parent_id           5800
type                5800
gender              5800
category_id         5800
dtype: int64

In [38]:
dataframe_2.isnull().sum()

competition_id      0
competition_name    0
parent_id           0
type                0
gender              0
category_id         0
dtype: int64

In [39]:
#CHECKING UNIQUE DATA
dataframe_2.nunique()


competition_id      5800
competition_name    5798
parent_id           2858
type                   4
gender                 3
category_id           16
dtype: int64

In [ ]:
#DISPLAY THE DATAFRAME AFTER CLEANING
dataframe_2

In [ ]:
#CONVERTING THE COMPETITIONS CLEANED DATAFRAME TO CSV FILES
dataframe_2.to_csv("Competitions_Table.csv",index=False)

SQL QUERY FOR COMPETITIONS AND CATEGORY TABLE

In [ ]:
#INSTALLING MYSQL CONNECTOR TO CONNECT SQL WITH PYTHON
pip install mysql-connector-python

In [41]:
#CONNECTING SQL WITH PYTHON 

import mysql.connector
connection= mysql.connector.connect(
 host= "localhost",
 user= "root",
 password="",
 database="tennis_project"
 )
print(connection)
mycursor = connection.cursor()

In [ ]:
#INSTALLING TABULATE FOR DISPLAY THE QUERY IN TABLE FORMAT
pip install tabulate

In [148]:
#add the foreign key to category_id in competition table
mycursor.execute("ALTER TABLE competitions_table ADD CONSTRAINT ID FOREIGN KEY (category_id) REFERENCES categories1_table(category_id)")

In [ ]:
#add the primary key to competition_id in competition table
mycursor.execute("ALTER TABLE competitions_table ADD CONSTRAINT COM PRIMARY KEY (competition_id)")

In [ ]:
#1)List all competitions along with their category name
mycursor.execute("""SELECT competitions_table.competition_name,categories1_table.category_name FROM competitions_table
                  INNER JOIN categories1_table ON competitions_table.category_id = categories1_table.category_id;""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#2)Count the number of competitions in each category
mycursor.execute("""SELECT category_name, COUNT(competition_name) AS competition_count
FROM (
    SELECT 
        competitions_table.competition_name, 
        categories1_table.category_name
    FROM competitions_table
    INNER JOIN categories1_table
    ON competitions_table.category_id = categories1_table.category_id
) AS combined_table
GROUP BY category_name ORDER BY competition_count""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#3) Find all competitions of type 'doubles'
mycursor.execute("select type,competition_name from competitions_table where type='doubles' ")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#4) Get competitions that belong to a specific category (e.g., ITF Men)
mycursor.execute("""select competition_name,category_name from (
                 SELECT 
                competitions_table.competition_name, 
                categories1_table.category_name
                FROM competitions_table
                INNER JOIN categories1_table
                ON competitions_table.category_id = categories1_table.category_id) AS combined2_table
                where category_name='challenger' """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#5) Identify parent competitions and their sub-competitions
mycursor.execute("SELECT parent_id, category_id from competitions_table")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#6) Analyze the distribution of competition types by category

mycursor.execute("SELECT category_id,competition_name, count(competition_name) as competition_count from competitions_table group by category_id")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql')) 

In [ ]:
#7) List all competitions with no parent (top-level competitions)
mycursor.execute("SELECT competition_name AS List_of_competions from competitions_table WHERE parent_id ='NA' ")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

SPORTS RADAR API COMPETITIONS DATA

In [ ]:
#COLLECTING JSON COMPLEXES DATA FROM SPORTS RADAR API
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=UMjToJVdpV74IvFAz1VrFWQcVMwv510g66U5yqOI"

headers = {"accept": "application/json"}

response1 = requests.get(url, headers=headers)

CREATING THE COMPLEXES TABLE

In [51]:
#STORING JSON DATA IN JSON_DATA VARIABLE
json1_data = response1.json()

Creating the Complexes Table

In [52]:
#CREATING EMPTY LIST TO STORE THE JSON DATA
complexes_list = []

In [53]:
#STORING THE DATA IN EMPTY LIST
for complexes in json1_data["complexes"]:
     complexes_info = {
          "complex_id": complexes.get("id"),
          "complex_name": complexes.get("name")
        
     }
     complexes_list.append(complexes_info)

In [54]:
#CREATING DATAFRAME FOR COMPLEXES DATA
dataframe_3 = pd.DataFrame(complexes_list)

In [ ]:
#DISPLAY THE DATAFRAME
dataframe_3

,complex_id,complex_name
0,sr:complex:705,Nacional
1,sr:complex:1078,Estadio de la Cartuja


In [56]:
#CHECKING NULL VALUES
dataframe_3.isnull().sum()

complex_id      0
complex_name    0
dtype: int64

In [57]:
#COUNTING THE DATA
dataframe_3.count()


complex_id      609
complex_name    609
dtype: int64

In [58]:
#CHECKING UNIQUE DATA
dataframe_3.nunique()

complex_id      609
complex_name    586
dtype: int64

In [59]:
#CONVERTING THE COMPLEX CLEANED DATAFRAME TO CSV FILES
dataframe_3.to_csv("Complexes_Table.csv",index=False)

CREATING THE COMPLEXES TABLE

In [60]:
#CREATING EMPTY LIST TO STORE THE JSON DATA
venues_list = []

In [61]:
#STORING THE DATA IN EMPTY LIST
for complexes in json1_data["complexes"]:
    for venues in complexes.get('venues', []):
        venue_info = {
            "venue_id": venues['id'],
            "venue_name": venues['name'],
            "city_name": venues['city_name'],
            "country_name": venues['country_name'],
            "country_code": venues['country_code'],
            "timezone": venues['timezone'],  
            "complex_id": complexes['id']
        }
        venues_list.append(venue_info)

In [62]:
#CREATING DATAFRAME FOR COMPLEXES DATA
dataframe_4 = pd.DataFrame(venues_list)

In [ ]:
#DISPLAY THE DATAFRAME
dataframe_4.head(20)

In [64]:
#CHECKING NULL VALUES
dataframe_4.isnull().sum()

venue_id        0
venue_name      0
city_name       0
country_name    0
country_code    0
timezone        0
complex_id      0
dtype: int64

In [65]:
#CHECKING UNIQUE DATA 
dataframe_4.nunique()

venue_id        3145
venue_name       619
city_name        343
country_name      64
country_code      64
timezone          79
complex_id       466
dtype: int64

In [66]:
#COUNTING THE DATA
dataframe_4.count()

venue_id        3145
venue_name      3145
city_name       3145
country_name    3145
country_code    3145
timezone        3145
complex_id      3145
dtype: int64

In [67]:
#CONVERTING THE CLEANED DATAFRAME TO CSV FILES
dataframe_4.to_csv("Venues_Table.csv",index =False)

SQL QUERY FOR COMPLEXES AND VENUES TABLE

In [68]:
#CONNECTING SQL WITH PYTHON 
import mysql.connector
connection= mysql.connector.connect(
 host= "localhost",
 user= "root",
 password="",
 database="tennis_project"
 )
print(connection)
mycursor = connection.cursor()

In [ ]:
#add the primary key to complex_id in complexes table
mycursor.execute("ALTER TABLE complexes_table ADD CONSTRAINT COMPLEX PRIMARY KEY (complex_id)")

In [88]:
#add the primary key to venue_id in venues table
mycursor.execute("ALTER TABLE venues_table ADD CONSTRAINT ven PRIMARY KEY (venue_id)")

In [ ]:
#add the foreign key to category_id in venues table
mycursor.execute("ALTER TABLE venues_table ADD CONSTRAINT COMPE FOREIGN KEY (complex_id) REFERENCES complexes_table(complex_id)")

In [ ]:
#1)List all venues along with their associated complex name
mycursor.execute("""SELECT venues_table.venue_name,complexes_table.complex_name FROM venues_table 
                 inner join complexes_table ON venues_table.complex_id=complexes_table.complex_id""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#2Count the number of venues in each complex
mycursor.execute(""" SELECT c.complex_name, COUNT(v.venue_name) AS venue_count
                     FROM venues_table v JOIN complexes_table c ON v.complex_id = c.complex_id
                     GROUP BY c.complex_name order by venue_count DESC ;
                     """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#3)Get details of venues in a specific country (e.g., Chile)
mycursor.execute("SELECT country_name,venue_id,venue_name,city_name,country_code,timezone from venues_table WHERE country_name='Australia' ")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
#4)Identify all venues and their timezones
mycursor.execute("SELECT venue_name, timezone FROM venues_table")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


+--------------------+---------------------+
| venue_name         | timezone            |
|--------------------+---------------------|
| Estadio Monumental | America/Lima        |
| Stadium            | Asia/Shanghai       |
| Court 2            | Asia/Shanghai       |
| Court 7            | Australia/Sydney    |
| Court 20           | Australia/Melbourne |
+--------------------+---------------------+


In [ ]:
#5)Find complexes that have more than one venue
mycursor.execute("""SELECT complex_id FROM (SELECT COUNT(venue_id) as venue_id_count, complex_id FROM venues_table GROUP BY complex_id) 
                 AS subquery
                  WHERE venue_id_count>1 """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
#6)List venues grouped by country
mycursor.execute("SELECT country_name, GROUP_CONCAT(venue_name) AS venues FROM venues_table GROUP BY country_name;")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [77]:
#7)Find all venues for a specific complex (e.g., Nacional)
mycursor.execute("""SELECT venue_name,complex_name FROM (SELECT venues_table.venue_name,complexes_table.complex_name FROM venues_table 
                 inner join complexes_table ON venues_table.complex_id=complexes_table.complex_id) AS joined_table WHERE complex_name='Nacional';""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))




+----------------+----------------+
| venue_name     | complex_name   |
|----------------+----------------|
| Cancha Central | Nacional       |
+----------------+----------------+


In [ ]:
#8) create the country count
mycursor.execute("SELECT COUNT(DISTINCT country_name) FROM venues_table")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+--------------------------------+
|   COUNT(DISTINCT country_name) |
|--------------------------------|
|                             64 |
+--------------------------------+


SPORTS RADAR API FOR Double_ranking_competitor 

In [ ]:
#COLLECTING JSON COMPETITROR RANKING DATA FROM SPORTS RADAR API
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=UMjToJVdpV74IvFAz1VrFWQcVMwv510g66U5yqOI"

headers = {"accept": "application/json"}

response2 = requests.get(url, headers=headers)

CREATING THE DOUBLE COMPETITOR RANKING TABLE

In [80]:
#STORING THE JSON DOUBLE COMPETITOR DATA IN JSON2 DATA
json2_data = response2.json()

In [81]:
#CREATING EMPTY LIST TO STORE THE JSON DATA
competitor_ranking_list = []

In [82]:
#STORING THE DATA IN EMPTY LIST
for ranks in json2_data["rankings"]:
    for competitor_ranking in ranks.get( "competitor_rankings",[]):
        competitor_ranking_info={
            "rank":competitor_ranking["rank"],
            "movement":competitor_ranking["movement"],
            "points":competitor_ranking["points"],
            "competitions_played":competitor_ranking["competitions_played"],
            "competitor_id":competitor_ranking["competitor"].get("id")
        }
        competitor_ranking_list.append( competitor_ranking_info)

In [83]:
#CREATING DATAFRAME FOR COMPLEXES DATA
dataframe_5 = pd.DataFrame(competitor_ranking_list)

In [ ]:
#DISPLAY THE DATAFRAME
dataframe_5.head(25)

In [85]:
#CHECKING NULL DATA
dataframe_5.isnull().sum()

rank                   0
movement               0
points                 0
competitions_played    0
competitor_id          0
dtype: int64

In [86]:
#CHECKING UNIQUE DATA
dataframe_5.nunique()

rank                    501
movement                 73
points                  563
competitions_played      45
competitor_id          1000
dtype: int64

In [87]:
#COUNTING THE DATA
dataframe_5.count()

rank                   1000
movement               1000
points                 1000
competitions_played    1000
competitor_id          1000
dtype: int64

In [88]:
#CONVERTING DATAFRAME TO CSV FILE
dataframe_5.to_csv("competitor_ranking_Table.csv",index=False)

CREATING THE COMPETITOR TABLE

In [89]:
#CREATING EMPTY LIST TO STORE THE DATA
competitor_list = []

In [90]:
#STORING THE DATA IN EMPTY LIST
for ranks in json2_data["rankings"]:
    for competitor_ranking in ranks.get( "competitor_rankings",[]):
        competitor_info={
            "competitor_id":competitor_ranking["competitor"].get("id"),
            "name":competitor_ranking["competitor"].get("name"),
            "country":competitor_ranking["competitor"].get("country"),
            "country_code":competitor_ranking["competitor"].get("country_code"),
            "abbreviation":competitor_ranking["competitor"].get("abbreviation")
        }
        competitor_list.append(competitor_info)

In [91]:
#CREATING DATAFRAME FOR COMPLEXES DATA
dataframe_6 = pd.DataFrame(competitor_list)

In [92]:
#DISPLAY THE COUNT
dataframe_6.count()

competitor_id    1000
name             1000
country          1000
country_code      931
abbreviation     1000
dtype: int64

In [93]:
#FILLING NULL VALUES 
dataframe_6.fillna('NAN',inplace=True)

In [ ]:
#DISPLAY THE DATAFRAME
dataframe_6.head(25)

In [95]:
#CHECKING NULL VALUE AFTER FILLING THE NULL DATA
dataframe_6.isnull().sum()

competitor_id    0
name             0
country          0
country_code     0
abbreviation     0
dtype: int64

In [96]:
#CHECKING UNIQUE DATA 
dataframe_6.nunique()

competitor_id    1000
name             1000
country            78
country_code       78
abbreviation      637
dtype: int64

In [97]:
#DISPLAY THE COUNT AFTER FILLING THE NULL DATA
dataframe_6.count()

competitor_id    1000
name             1000
country          1000
country_code     1000
abbreviation     1000
dtype: int64

In [ ]:
#CONVERTING COMPETITORS DATAFRAME TO CSV FILE
dataframe_6.to_csv("Competitor_table.csv",index=False)

SQL QUERY FOR COMPLEXES AND VENUES TABLE

In [98]:
#CONNECTING SQL WITH PYTHON
import mysql.connector
connection= mysql.connector.connect(
 host= "localhost",
 user= "root",
 password="",
 database="tennis_project"
 )
print(connection)
mycursor = connection.cursor()

In [ ]:
#CREATING RANK ID AND MAKE AS A PRIMARY KEY
mycursor.execute("ALTER TABLE competitor_ranking_table ADD COLUMN rank_id INT AUTO_INCREMENT PRIMARY KEY FIRST ")

In [ ]:
# MAKE COMPETITOR ID AS A PRIMARY KEY
mycursor.execute("ALTER TABLE competitor_table ADD CONSTRAINT COMPET PRIMARY KEY(competitor_id)")

In [ ]:
#CREATING FOREIGN KEY AS COMPETITOR ID
mycursor.execute("""
    ALTER TABLE competitor_ranking_table 
    ADD CONSTRAINT RANK FOREIGN KEY (competitor_id) 
    REFERENCES competitor_table (competitor_id)
""")

In [ ]:
#1)Get all competitors with their rank and points.
mycursor.execute("""SELECT competitor_table.name,competitor_ranking_table.rank,competitor_ranking_table.points
                FROM competitor_table inner join competitor_ranking_table ON 
                 competitor_table.competitor_id=competitor_ranking_table.competitor_id order by points DESC""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [100]:
#2)Find competitors ranked in the top 5
mycursor.execute("""SELECT competitor_table.name,competitor_ranking_table.rank
                FROM competitor_table inner join competitor_ranking_table ON 
                 competitor_table.competitor_id=competitor_ranking_table.competitor_id ORDER BY rank LIMIT 5""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+---------------------------+--------+
| name                      |   rank |
|---------------------------+--------|
| Pavic, Mate               |      1 |
| Arevalo-Gonzalez, Marcelo |      1 |
| Siniakova, Katerina       |      1 |
| Routliffe, Erin           |      2 |
| Thompson, Jordan          |      3 |
+---------------------------+--------+


In [ ]:
#3)List competitors with no rank movement (stable rank)
mycursor.execute("""SELECT competitor_table.name, competitor_ranking_table.rank
FROM competitor_table 
INNER JOIN competitor_ranking_table 
    ON competitor_table.competitor_id = competitor_ranking_table.competitor_id
WHERE competitor_ranking_table.rank = competitor_ranking_table.rank
ORDER BY rank""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))



In [102]:
#4)Get the total points of competitors from a specific country (e.g., Croatia)
mycursor.execute("""SELECT  country, sum(points) AS total_points FROM (SELECT competitor_table.country,competitor_ranking_table.points
                FROM competitor_table inner join competitor_ranking_table ON 
                 competitor_table.competitor_id=competitor_ranking_table.competitor_id) AS NEWTABLE WHERE country = 'Croatia' """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+-----------+----------------+
| country   |   total_points |
|-----------+----------------|
| Croatia   |          17031 |
+-----------+----------------+


In [ ]:
#5)Count the number of competitors per country
mycursor.execute("SELECT country,COUNT(name) AS competitor_count from competitor_table group by country order by competitor_count DESC")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [104]:
#6.Find competitors with the highest points in the current week
mycursor.execute("SELECT max(points) AS MAX_POINT_in_current_week from competitor_ranking_table")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


+-----------------------------+
|   MAX_POINT_in_current_week |
|-----------------------------|
|                        9530 |
+-----------------------------+


In [105]:
#7.Find competitors with the highest points in the current week
mycursor.execute("SELECT AVG(points) AS AVG_POINT from competitor_ranking_table")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+-------------+
|   AVG_POINT |
|-------------|
|      722.16 |
+-------------+
